In [2]:
import requests, json, urllib.parse

In [23]:
base_url = 'https://itunes.apple.com/search'

artist = 'Gorillaz'

params = {
    'term': artist,
    'limit': 152,
    'media': 'music',
    'entity': 'song',
    'attribute': 'artistTerm'
}

response = requests.get(f"{base_url}?{urllib.parse.urlencode(params)}")

data = json.loads(response.text)
data = data['results']

In [24]:
data = map(lambda x: {
    'artistName': x['artistName'],
    'kind' : x['kind'],
    'artistId': x['artistId'],
    'collectionId': x['collectionId'],
    'trackId': x['trackId'],
    'artistName': x['artistName'],
    'collectionName': x['collectionName'],
    'trackName': x['trackName'],
    'trackPrice': x['trackPrice'],
    'releaseDate': x['releaseDate'],
    'trackCount': x['trackCount'],
    'trackTimeMillis': x['trackTimeMillis'],
    'primaryGenreName': x['primaryGenreName']
}, data)
data = list(data)

In [25]:
def convert_millis(millis: int) -> str:
    total_seconds = millis // 1000
    seconds = total_seconds % 60
    minutes = (total_seconds // 60) % 60
    hours = (total_seconds // (60 * 60))
    return f"{hours}:Hours,{minutes}:Minutes,{seconds}:Seconds"

In [26]:
def get_info_songs(data):
    result = {}
    result['Count of tracks'] = len(data)
    result['Most popular genre'] = statistics.mode(list(map(lambda x: x['primaryGenreName'].lower(), data)))
    result['Count of collection'] = len(set(map(lambda x: x['collectionId'], data)))

    result['Total track time'] = sum(list(map(lambda x: x['trackTimeMillis'], data)))
    result['Average track time'] = convert_millis(result['Total track time'] / result['Count of tracks'])
    result['Median track time'] = convert_millis(statistics.median(map(lambda x: x['trackTimeMillis'], data)))
    result['Total track time'] = convert_millis(result['Total track time'])

    result['Longest song time'] = max(map(lambda x: x['trackTimeMillis'], data))
    result['Longest song name'] = next(filter(lambda x: x.get('trackTimeMillis') == result['Longest song time'], data), None)['trackName']
    result['Longest song time'] = convert_millis(result['Longest song time'])

    result['Shortest song time'] = min(map(lambda x: x['trackTimeMillis'], data))
    result['Shortest song name'] = next(filter(lambda x: x.get('trackTimeMillis') == result['Shortest song time'], data), None)['trackName']
    result['Shortest song time'] = convert_millis(result['Shortest song time'])

    result['Total cost of songs'] = sum(list(map(lambda x: x['trackPrice'], data)))
    result['Average cost of songs'] = result['Total cost of songs'] / result['Count of tracks']
    result['Median cost of songs'] = statistics.median(map(lambda x: x['trackPrice'], data))

    result['Most expensive song cost'] = max(map(lambda x: x['trackPrice'], data))
    result['Most expensive song name'] = next(filter(lambda x: x.get('trackPrice') == result['Most expensive song cost'], data), None)['trackName']

    result['Сheapest song cost'] = min(map(lambda x: x['trackPrice'], data))
    result['Сheapest song name'] = next(filter(lambda x: x.get('trackPrice') == result['Сheapest song cost'], data), None)['trackName']
    return result

In [27]:
import statistics

info = get_info_songs(data)
info['Artist'] = artist
info

{'Count of tracks': 152,
 'Most popular genre': 'alternative',
 'Count of collection': 18,
 'Total track time': '9:Hours,18:Minutes,44:Seconds',
 'Average track time': '0.0:Hours,3.0:Minutes,40.0:Seconds',
 'Median track time': '0.0:Hours,3.0:Minutes,40.0:Seconds',
 'Longest song time': '0:Hours,12:Minutes,14:Seconds',
 'Longest song name': 'DARE (DFA Remix)',
 'Shortest song time': '0:Hours,0:Minutes,4:Seconds',
 'Shortest song name': 'Interlude: Elevator Going Up',
 'Total cost of songs': 180.04999999999993,
 'Average cost of songs': 1.18453947368421,
 'Median cost of songs': 1.29,
 'Most expensive song cost': 1.29,
 'Most expensive song name': 'Feel Good Inc.',
 'Сheapest song cost': -1.0,
 'Сheapest song name': 'Three Hearts, Seven Seas, Twelve Moons',
 'Artist': 'Gorillaz'}

In [28]:
from itertools import groupby

collection_id_extractor = lambda x: x['collectionId']
album_groups = {}
album_names = {}

sorted_by_album = sorted(data, key=collection_id_extractor)
for album_id, songs in groupby(sorted_by_album, key=collection_id_extractor):
    songs = list(songs)
    album_groups[album_id] = songs 
    album_names[album_id] = songs[0]['collectionName']

info_album_list = []
for album_id, songs in album_groups.items():
    info_album = get_info_songs(songs)
    info_album['Album name'] = album_names[album_id]
    info_album['Release date album'] = album_groups[album_id][0]['releaseDate']
    info_album_list.append(info_album)
info_album_list 

[{'Count of tracks': 1,
  'Most popular genre': 'soundtrack',
  'Count of collection': 1,
  'Total track time': '0:Hours,3:Minutes,45:Seconds',
  'Average track time': '0.0:Hours,3.0:Minutes,45.0:Seconds',
  'Median track time': '0:Hours,3:Minutes,45:Seconds',
  'Longest song time': '0:Hours,3:Minutes,45:Seconds',
  'Longest song name': 'Kids With Guns',
  'Shortest song time': '0:Hours,3:Minutes,45:Seconds',
  'Shortest song name': 'Kids With Guns',
  'Total cost of songs': -1.0,
  'Average cost of songs': -1.0,
  'Median cost of songs': -1.0,
  'Most expensive song cost': -1.0,
  'Most expensive song name': 'Kids With Guns',
  'Сheapest song cost': -1.0,
  'Сheapest song name': 'Kids With Guns',
  'Album name': 'The O.C. Mix 5',
  'Release date album': '2005-05-11T12:00:00Z'},
 {'Count of tracks': 1,
  'Most popular genre': 'hip-hop/rap',
  'Count of collection': 1,
  'Total track time': '0:Hours,3:Minutes,42:Seconds',
  'Average track time': '0.0:Hours,3.0:Minutes,42.0:Seconds',
  '